In [7]:
import json
import pandas as pd
from urllib.parse import urlparse

In [9]:
bulk_data = [
    {
        "data": {
            "collection": 85,
            "url": "exo.mast.stsci.edu/docs/",
            "excluded": False,
            "title": "Root URL is this one",
        },
        "children": []
    }
]

def add_to_root_node(path, title, root_node):
    """Recursively figure out where to add a URL to the root node."""
    path_parts = path.split('/')
    if path_parts[0] == '':
        path_parts = path_parts[1:]
    if path_parts[0] == '':
        path_parts = path_parts[1:]
    if path_parts[-1] == '':
        path_parts = path_parts[:-1]
    if path_parts[-1] == '':
        path_parts = path_parts[:-1]
    if len(path_parts) == 0:
        return
    if len(path_parts) == 1:
        root_node['children'].append({
            "data": {
                "collection": 85,
                "url": path,
                "excluded": False,
                "title": title,
            },
        })
        return
    for child in root_node['children']:
        if child['data']['url'] == path_parts[0]:
            add_to_root_node('/'.join(path_parts[1:]), title, child)
            return
    new_node = {
        "data": {
            "collection": 85,
            "url": path_parts[0],
            "excluded": False,
            "title": path_parts[0],
        },
        "children": []
    }
    root_node['children'].append(new_node)
    add_to_root_node('/'.join(path_parts[1:]), title, new_node)

with open('urls.jsonl', 'r') as jsonfile:
    urls = jsonfile.readlines()
    root_node = {
        "data": {
            "collection": 85,
            "url": json.loads(urls[0])['url'],
            "excluded": False,
            "title": json.loads(urls[0])['title'],
        },
        "children": []
    }
    bulk_data.append(root_node)

    for url in urls[1:]:
        url = json.loads(url)
        parsed = urlparse(url['url'])
        add_to_root_node(parsed.path, url['title'], root_node)
        break

/docs/spectra_ws.html


In [12]:
path = '/dvdata/kepler/8394721/phaseplot/?tce=2'
path.split('/')

['', 'dvdata', 'kepler', '8394721', 'phaseplot', '?tce=2']

In [5]:
url

{'url': 'https://exo.mast.stsci.edu/api/v0.1/dvdata/kepler/8394721/phaseplot/?tce=2',
 'title': 'Phased Light Curve'}

In [26]:
from anytree import Node, RenderTree

root = Node('root')

def add_to_tree(path, parent=root):
    splits = path.split('/')
    if splits[0] == '':
        parent = root
    
    for split in splits:
        node = Node(split, parent=parent)
        add_to_tree('/'.join(splits[1:]), parent=node)

with open('urls.jsonl', 'r') as jsonfile:
    urls = jsonfile.readlines()

    root = Node('root')
    for url in urls:
        url = json.loads(url)
        parsed = urlparse(url['url'])
        add_to_tree(parsed.path)

print(RenderTree(root))

Node('/root')


In [27]:
with open('urls.jsonl', 'r') as jsonfile:
    urls = jsonfile.readlines()

In [36]:
lst = [urlparse(json.loads(url)['url']).path.split('/')[1:] for url in urls]

In [38]:
from collections import defaultdict

tree = lambda: defaultdict(tree)

def make_tree(lst):
    d = tree()    
    for x in lst:
        curr = d
        for item in x:
             curr = curr[item]
    return d

d = make_tree(lst)

def make_strs(d, indent=0):
     strs = []
     for k, v in d.items():
         strs.append('    ' * indent + str(k))
         strs.extend(make_strs(v, indent+1))
     return strs

def print_tree(d):
    print('\n'.join(make_strs(d)))

# print_tree(d)

In [55]:
def make_bulk_data(d, level=0):
    children = []
    for k, v in d.items():
        if not k:
            continue
        children.append({"data": {"url": k,}, "children": make_bulk_data(v, level+1)})
    return children

make_bulk_data(d)

[{'data': {'url': 'docs'},
  'children': [{'data': {'url': 'spectra_ws.html'}, 'children': []},
   {'data': {'url': '_sources'},
    'children': [{'data': {'url': 'index.rst.txt'}, 'children': []},
     {'data': {'url': 'spectra_ws.rst.txt'}, 'children': []},
     {'data': {'url': 'exoplanets_ws.rst.txt'}, 'children': []},
     {'data': {'url': 'swagger'},
      'children': [{'data': {'url': 'index.rst.txt'}, 'children': []}]},
     {'data': {'url': 'dvdata_ws.rst.txt'}, 'children': []},
     {'data': {'url': 'getting_started.rst.txt'}, 'children': []}]},
   {'data': {'url': 'exoplanets_ws.html'}, 'children': []},
   {'data': {'url': 'dvdata_ws.html'}, 'children': []},
   {'data': {'url': 'swagger'},
    'children': [{'data': {'url': 'index.html'}, 'children': []}]},
   {'data': {'url': 'getting_started.html'}, 'children': []},
   {'data': {'url': 'index.html'}, 'children': []}]},
 {'data': {'url': 'api'},
  'children': [{'data': {'url': 'v0.1'},
    'children': [{'data': {'url': 'dvda